In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [0]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [12]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [13]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 18s - loss: 0.8278 - accuracy: 0.6416
Epoch 2/20
 - 16s - loss: 0.6826 - accuracy: 0.7082
Epoch 3/20
 - 16s - loss: 0.6228 - accuracy: 0.7388
Epoch 4/20
 - 16s - loss: 0.5750 - accuracy: 0.7625
Epoch 5/20
 - 16s - loss: 0.5294 - accuracy: 0.7825
Epoch 6/20
 - 16s - loss: 0.4938 - accuracy: 0.8002
Epoch 7/20
 - 16s - loss: 0.4549 - accuracy: 0.8135
Epoch 8/20
 - 16s - loss: 0.4259 - accuracy: 0.8297
Epoch 9/20
 - 16s - loss: 0.3950 - accuracy: 0.8386
Epoch 10/20
 - 16s - loss: 0.3746 - accuracy: 0.8455
Epoch 11/20
 - 16s - loss: 0.3455 - accuracy: 0.8598
Epoch 12/20
 - 16s - loss: 0.3310 - accuracy: 0.8645
Epoch 13/20
 - 16s - loss: 0.3178 - accuracy: 0.8707
Epoch 14/20
 - 16s - loss: 0.3005 - accuracy: 0.8778
Epoch 15/20
 - 16s - loss: 0.2912 - accuracy: 0.8809
Epoch 16/20
 - 16s - loss: 0.2741 - accuracy: 0.8875
Epoch 17/20
 - 16s - loss: 0.2561 - accuracy: 0.8947
Epoch 18/20
 - 16s - loss: 0.2540 - accuracy: 0.8944
Epoch 19/20
 - 16s - loss: 0.2474 - accuracy: 0.9011
Ep

In [0]:
model.save('Bhuvana.h5')

In [15]:
from keras.models import load_model
model= load_model('Bhuvana.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [16]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


2. GridSearchCV 

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 43s - loss: 0.8252 - accuracy: 0.6489
Epoch 1/1
 - 40s - loss: 0.8249 - accuracy: 0.6462
Epoch 1/1
 - 40s - loss: 0.8292 - accuracy: 0.6426
Epoch 1/1
 - 39s - loss: 0.8326 - accuracy: 0.6418
Epoch 1/1
 - 40s - loss: 0.8264 - accuracy: 0.6461
Epoch 1/2
 - 40s - loss: 0.8323 - accuracy: 0.6427
Epoch 2/2
 - 40s - loss: 0.6902 - accuracy: 0.7082
Epoch 1/2
 - 39s - loss: 0.8239 - accuracy: 0.6474
Epoch 2/2
 - 41s - loss: 0.6907 - accuracy: 0.7120
Epoch 1/2
 - 43s - loss: 0.8298 - accuracy: 0.6400
Epoch 2/2
 - 41s - loss: 0.6848 - accuracy: 0.7100
Epoch 1/2
 - 40s - loss: 0.8305 - accuracy: 0.6387
Epoch 2/2
 - 40s - loss: 0.6895 - accuracy: 0.6997
Epoch 1/2
 - 40s - loss: 0.8210 - accuracy: 0.6477
Epoch 2/2
 - 40s - loss: 0.6726 - accuracy: 0.7138
Epoch 1/1
 - 20s - loss: 0.8472 - accuracy: 0.6338
Epoch 1/1
 - 21s - loss: 0.8285 - accuracy: 0.6461
Epoch 1/1
 - 20s - loss: 0.8404 - accuracy: 0.6345
Epoch 1/1
 - 20s - loss: 0.8343 - accuracy: 0.6362
Epoch 1/1
 - 20s - loss: 0.8266

3. spamdata set

In [0]:
data1 = pd.read_csv('spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [24]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 32s - loss: 0.1912 - accuracy: 0.9387
Epoch 2/5
 - 31s - loss: 0.0421 - accuracy: 0.9866
Epoch 3/5
 - 32s - loss: 0.0267 - accuracy: 0.9914
Epoch 4/5
 - 31s - loss: 0.0139 - accuracy: 0.9965
Epoch 5/5
 - 32s - loss: 0.0072 - accuracy: 0.9979
0.09370256584216681
0.9728112816810608
['loss', 'accuracy']
